In [1]:
import os
os.environ["MLFLOW_TRACKING_URI"] = "http://127.0.0.1:5000"

In [2]:
# Import des bibliothèques nécessaires
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV

In [3]:
# Charger les données prétraitées
app_train = pd.read_csv("Base De Donnée Prétraitée.csv")
app_test = pd.read_csv("Base De Donnée Prétraitée Test.csv")

In [4]:
# Séparer les caractéristiques et la cible
if 'TARGET' in app_train:
    train_labels = app_train['TARGET']
    train = app_train.drop(columns=['TARGET'])
else:
    train = app_train.copy()
    
# Préparation des données de test
test = app_test.copy()

# Imputation médiane des valeurs manquantes
imputer = SimpleImputer(strategy='median')
scaler = MinMaxScaler(feature_range=(0, 1))

# Ajuster l'imputer sur les données d'entraînement et transformer
train = imputer.fit_transform(train)
test = imputer.transform(test)

# Ajuster le scaler sur les données d'entraînement et transformer
train = scaler.fit_transform(train)
test = scaler.transform(test)

# Séparation des données d'entraînement pour validation
X_train, X_test, y_train, y_test = train_test_split(train, train_labels, test_size=0.3, random_state=0)

In [5]:
# Définir la grille des hyperparamètres
param_grid = {
    'C': [0.0001, 0.001, 0.01, 0.1, 1, 10],
    'solver': ['liblinear', 'lbfgs']
}

In [6]:
# Initialiser le modèle
log_reg = LogisticRegression(random_state=0)

In [7]:
# Configurer le GridSearchCV
grid_search = GridSearchCV(estimator=log_reg, 
                           param_grid=param_grid, 
                           scoring='roc_auc', 
                           cv=5, 
                           n_jobs=-1, 
                           verbose=1)

In [8]:
# Démarrer une expérimentation MLFlow
with mlflow.start_run():
    # Exécuter la recherche en grille
    grid_search.fit(X_train, y_train)
    
    # Obtenir les meilleurs paramètres et les meilleures performances
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    # Entraîner le meilleur modèle sur l’ensemble d’entraînement complet
    best_model = grid_search.best_estimator_
    best_model.fit(X_train, y_train)
    
    # Évaluation sur l’ensemble de test
    y_pred = best_model.predict(X_test)
    y_proba = best_model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_proba)
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    fp = conf_matrix[0][1]
    fn = conf_matrix[1][0]
    cost = fp + 10 * fn

    # Enregistrement des métriques et paramètres dans MLFlow
    mlflow.log_params(best_params)
    mlflow.log_metric("Best CV AUC", best_score)
    mlflow.log_metric("Test AUC", auc)
    mlflow.log_metric("Test Accuracy", accuracy)
    mlflow.log_metric("Test Cost", cost)
    mlflow.log_param("model_type", "LogisticRegression")

    # Loguer le meilleur modèle
    mlflow.sklearn.log_model(best_model, "best_logistic_regression_model")

    # Afficher les résultats
    print("Best Parameters found by GridSearchCV:", best_params)
    print(f"Best CV AUC Score: {best_score}")
    print(f"Test AUC: {auc}")
    print(f"Test Accuracy: {accuracy}")
    print(f"Test Cost: {cost}")


Fitting 5 folds for each of 12 candidates, totalling 60 fits


/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

Best Parameters found by GridSearchCV: {'C': 10, 'solver': 'liblinear'}
Best CV AUC Score: 0.7465902314585946
Test AUC: 0.7485621372447766
Test Accuracy: 0.9194614867648015
Test Cost: 73220


- Tester tous les modèles sans gridsearchCV (1 ou 2 par grandes familles)
- Selectionner les meilleurs modèles car ce ne sont pas les hyperparamètres qui changent beaucoup. 
- Tester les valeurs extrêmes des hyperparamètres pour savoir dans quel direction aller. 
- Ensuite on peut choisir le meilleur modèle et faire évoluer les hyperparamètres. 
- Un fois que le gridsearchCV est terminé, on n'en fait plus et on reste sur les valeurs selectionnées. 
- Il faut trouver trouver des modèles avec un features importance globale et locale (pas de blackbox)

Pour le déséquilibre, regarder les modèles qui intègrent les déséquilibres. Ou alors, créer des faux clients qui n'auraient pas et de crétit (regarder SMOTE) Data Augmentation

DataDRIFT
Introduction de la notion de temps en se plaçant dans le passé. Se mettre comme si on était en 2022 par exemple. 